In [0]:
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import time

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM

from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer

# Datasets
##Cat_versus_dog

In [9]:
catdog_builder = tfds.builder("cats_vs_dogs")
catdog_builder.download_and_prepare()

catdog_train_dataset, catdog_test_dataset = catdog_builder.as_dataset(
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True
)

catdog_num_classes = catdog_builder.info.features['label'].num_classes
catdog_data_type = 'text' if 'text' in catdog_builder.info.features else 'image'

catdog_train_dataset = catdog_train_dataset.map(lambda img, label: (tf.image.resize(img, [64, 64]) / 255.0, label))
catdog_test_dataset = catdog_test_dataset.map(lambda img, label: (tf.image.resize(img, [64, 64]) / 255.0, label))

print(tf.data.experimental.cardinality(catdog_train_dataset))
print(tf.data.experimental.cardinality(catdog_test_dataset))
print(catdog_num_classes)
print(catdog_data_type)

tf.Tensor(18610, shape=(), dtype=int64)
tf.Tensor(4652, shape=(), dtype=int64)
2
image


##Flowers

In [10]:
flower_builder = tfds.builder("tf_flowers")
flower_builder.download_and_prepare()

flower_train_dataset, flower_test_dataset = flower_builder.as_dataset(
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True
)

flower_num_classes = flower_builder.info.features['label'].num_classes
flower_data_type = 'text' if 'text' in flower_builder.info.features else 'image'

flower_train_dataset = flower_train_dataset.map(lambda img, label: (tf.image.resize(img, [64, 64]) / 255.0, label))
flower_test_dataset = flower_test_dataset.map(lambda img, label: (tf.image.resize(img, [64, 64]) / 255.0, label))

print(tf.data.experimental.cardinality(flower_train_dataset))
print(tf.data.experimental.cardinality(flower_test_dataset))
print(flower_num_classes)
print(flower_data_type)

tf.Tensor(2936, shape=(), dtype=int64)
tf.Tensor(734, shape=(), dtype=int64)
5
image


##Malaria

In [11]:
malaria_builder = tfds.builder("malaria")
malaria_builder.download_and_prepare()

malaria_train_dataset, malaria_test_dataset = malaria_builder.as_dataset(
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True
)

malaria_num_classes = malaria_builder.info.features['label'].num_classes
malaria_data_type = 'text' if 'text' in malaria_builder.info.features else 'image'

malaria_train_dataset = malaria_train_dataset.map(lambda img, label: (tf.image.resize(img, [64, 64]) / 255.0, label))
malaria_test_dataset = malaria_test_dataset.map(lambda img, label: (tf.image.resize(img, [64, 64]) / 255.0, label))

print(tf.data.experimental.cardinality(malaria_train_dataset))
print(tf.data.experimental.cardinality(malaria_test_dataset))
print(malaria_num_classes)
print(malaria_data_type)

tf.Tensor(22046, shape=(), dtype=int64)
tf.Tensor(5512, shape=(), dtype=int64)
2
image


# Model

In [0]:
def compute_original_epsilon():
  """Computes epsilon value of Gaussian Mechanism"""
  orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
  noise_multiplier = 1.1
  sampling_probability = 1
  rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=noise_multiplier,
                    steps=1,
                    orders=orders)
  return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]

In [0]:
def compute_shuffle_epsilon(epochs, batch_size, dataset_size):
  """Computes epsilon value of shuffle"""
  delta = 1e-5
  shuffle_epsilon = compute_original_epsilon()
  composition_epsilon = shuffle_epsilon*np.sqrt(2*epochs*np.log(1/delta)) + epochs*shuffle_epsilon*(np.exp(shuffle_epsilon)-1)
  return composition_epsilon

In [0]:
def compute_swo_epsilon(steps, batch_size, dataset_size):
  """Computes epsilon value of swo"""
  delta = 1e-5
  epsilon = compute_original_epsilon()
  sampling_probability = batch_size / dataset_size
  swo_epsilon = np.log(1+sampling_probability*(np.exp(epsilon)-1))
  composition_epsilon = swo_epsilon*np.sqrt(2*steps*np.log(1/delta)) + steps*swo_epsilon*(np.exp(swo_epsilon)-1)
  return composition_epsilon

In [0]:
def compute_poisson_epsilon(steps, batch_size, dataset_size):
  """Computes epsilon value for Poisson"""
  orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
  noise_multiplier = 1.1
  sampling_probability = batch_size / dataset_size
  rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=noise_multiplier,
                    steps=steps,
                    orders=orders)
  return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]

# Shuffling

##Cat_versus_dog

In [20]:
train_dataset_size = int(tf.data.experimental.cardinality(catdog_train_dataset))
eps = compute_shuffle_epsilon(10, 50, train_dataset_size)
print('For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: %.2f' % eps)

For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: 5687.38


##Flowers

In [21]:
train_dataset_size = int(tf.data.experimental.cardinality(flower_train_dataset))
eps = compute_shuffle_epsilon(10, 50, train_dataset_size)
print('For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: %.2f' % eps)

For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: 5687.38


##Malaria

In [22]:
train_dataset_size = int(tf.data.experimental.cardinality(malaria_train_dataset))
eps = compute_shuffle_epsilon(10, 50, train_dataset_size)
print('For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: %.2f' % eps)

For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: 5687.38


# SWO

##Cat_versus_dog

In [0]:
train_dataset_size = int(tf.data.experimental.cardinality(catdog_train_dataset))
eps = compute_swo_epsilon(10*train_dataset_size//50, 50, train_dataset_size)
print('For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: %.2f' % eps)

For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: 403.13


##Flowers

In [0]:
train_dataset_size = int(tf.data.experimental.cardinality(flower_train_dataset))
eps = compute_swo_epsilon(10*train_dataset_size//50, 50, train_dataset_size)
print('For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: %.2f' % eps)

For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: 1419.99


##Malaria

In [0]:
train_dataset_size = int(tf.data.experimental.cardinality(malaria_train_dataset))
eps = compute_swo_epsilon(10*train_dataset_size//50, 50, train_dataset_size)
print('For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: %.2f' % eps)

For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: 353.24


# Poisson

##Cat_versus_dog

In [0]:
train_dataset_size = int(tf.data.experimental.cardinality(catdog_train_dataset))
eps = compute_poisson_epsilon(10*train_dataset_size//50, 50, train_dataset_size)
print('For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: %.2f' % eps)

For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: 1.20


##Flowers

In [0]:
train_dataset_size = int(tf.data.experimental.cardinality(flower_train_dataset))
eps = compute_poisson_epsilon(10*train_dataset_size//50, 50, train_dataset_size)
print('For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: %.2f' % eps)

For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: 2.78


##Malaria

In [0]:
train_dataset_size = int(tf.data.experimental.cardinality(malaria_train_dataset))
eps = compute_poisson_epsilon(10*train_dataset_size//50, 50, train_dataset_size)
print('For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: %.2f' % eps)

For delta_1=1e-5 and delta_2=1e-5, the current epsilon is: 1.12
